In [21]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score
import utils
import numpy as np
from sklearn import tree
import matplotlib.pyplot as plt

In [2]:
def filter_by_prefix(df,prefix):
    '''
    Filter case by prefix length
    
    Parameters
    ----------
    df : pandas dataframe
        Assigned dataframe to slice by prefix length
    
    prefix : int
        Prefix length to slice to cases in fixed length
    
    Returns
    ----------
    Return dataframe with sliced cases
    '''
    df['ts'] = pd.to_datetime(df['ts'])
    groups = df.groupby('caseid')
    encoded_df=[]
    for case,group in groups: 
        group = group.reset_index(drop=True)
        if len(group)>prefix:
            group = group.loc[:prefix-1,:]
            encoded_df.append(group)
    return pd.concat(encoded_df)

In [3]:
def aggregation_encoding(df, prefix):
    '''
    Aggregation encoding
    
    Parameters
    ----------
    df : pandas dataframe
        Assigned dataframe to encode for outcome prediction
    
    prefix : int
        Prefix length to slice to cases in fixed length
    
    Returns
    ----------
    Return dataframe encoded in aggregation method
    '''
    df = filter_by_prefix(df,prefix)
    df['ts'] = pd.to_datetime(df['ts'])
    groups = df.groupby('caseid')
    encoded_df=[]
    for case,group in groups: 
        group = group.reset_index(drop=True)
        outcome = set(group['outcome']).pop()
        cumdurationlist = [(x - list(group['ts'])[0]).total_seconds() for x in list(group['ts'])]
        case_time_outcome = {'caseid':case, 'ts':np.mean(cumdurationlist),'outcome':outcome}
        activity_count = {x: list(group['activity']).count(x) for x in set(group['activity'])}
        resource_count = {x: list(group['resource']).count(x) for x in set(group['resource'])}

        case_time_outcome.update(activity_count)
        case_time_outcome.update(resource_count)
        dfk = pd.DataFrame.from_dict([case_time_outcome])
        encoded_df.append(dfk)
    concated_df = pd.concat(encoded_df)
    concated_df = concated_df.fillna(0)
    return concated_df

In [4]:
def indexbase_encoding(df, prefix):
    '''
    Indexbase encoding
    
    Parameters
    ----------
    df : pandas dataframe
        Assigned dataframe to encode for outcome prediction
    
    prefix : int
        Prefix length to slice to cases in fixed length
    
    Returns
    ----------
    Return dataframe encoded in indexbase method
    '''
    df = filter_by_prefix(df,prefix)
    df['ts'] = pd.to_datetime(df['ts'])
    groups = df.groupby('caseid')
    encoded_df=[]
    for case,group in groups: 
        activitylist = list(group['activity'])
        resourcelist = list(group['resource'])
        group = group.reset_index(drop=True)
        outcome = set(group['outcome']).pop()
        cumdurationlist = [(x - list(group['ts'])[0]).total_seconds() for x in list(group['ts'])]
        cumduration_index ={'Cumduration_'+str(x+1): cumdurationlist[x] for x in range(len(cumdurationlist))}
        case_outcome = {'caseid':case, 'outcome':outcome}
        activity_index = {'activity_'+str(x+1)+'_'+activitylist[x]: 1 for x in range(len(activitylist))}
        resource_index = {'resource_'+str(x+1)+'_'+str(resourcelist[x]): 1 for x in range(len(resourcelist))}
        case_outcome.update(cumduration_index)
        case_outcome.update(activity_index)
        case_outcome.update(resource_index)
        dfk = pd.DataFrame.from_dict([case_outcome])
        encoded_df.append(dfk)
    concated_df = pd.concat(encoded_df)
    concated_df = concated_df.fillna(0)
    return concated_df

In [5]:
df = pd.read_csv('./data/bac_online_small.csv')
df['START_DATE'] = pd.to_datetime(df['START_DATE'])
df = df.rename(columns={'REQUEST_ID':'caseid','ACTIVITY':'activity','START_DATE':'ts','CE_UO':'resource'})
df = df.loc[:,['caseid','activity','ts','resource','outcome']]

In [6]:
groups = df.groupby('caseid')
concating = []
for _, group in groups:
    outcomelist = list(group['outcome'])
    outcome = outcomelist[-1]
    group = group.reset_index(drop=True)
    if True in outcomelist:
        group = group.loc[:outcomelist.index(True),:]
    group['outcome'] = outcome
    concating.append(group)

dfn = pd.concat(concating)

In [7]:
adf10 = aggregation_encoding(dfn,10)
idf10 = indexbase_encoding(dfn,10)

In [8]:
adf5 = aggregation_encoding(dfn,5)
idf5 = indexbase_encoding(dfn,5)

In [10]:
adf6 = aggregation_encoding(dfn,6)


In [11]:
adf7 = aggregation_encoding(dfn,7)


In [12]:
adf8 = aggregation_encoding(dfn,8)


In [13]:
adf9 = aggregation_encoding(dfn,9)


In [27]:
adf11 = aggregation_encoding(dfn,11)

In [28]:
adf2 = aggregation_encoding(dfn,2)
adf3 = aggregation_encoding(dfn,3)
adf4 = aggregation_encoding(dfn,4)

In [22]:
print('BAC aggregation encoding with prefix length 5')
y = adf5['outcome']
x =adf5.drop(columns=['outcome','caseid'],axis=1)
x_train,x_test,y_train,y_test = train_test_split(x,y, test_size=0.3)

# Deicision tree result
print('Decision Tree')
dt = DecisionTreeClassifier(criterion='entropy', max_depth=5).fit(x_train,y_train)
y_pred = dt.predict(x_test)
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

rf = RandomForestClassifier(criterion='entropy', max_depth=5).fit(x_train,y_train)
y_pred = rf.predict(x_test)
print(classification_report(y_test,y_pred))


BAC aggregation encoding with prefix length 5
Decision Tree
0.9227251943301326
              precision    recall  f1-score   support

       False       0.94      0.98      0.96      2006
        True       0.55      0.34      0.42       181

    accuracy                           0.92      2187
   macro avg       0.75      0.66      0.69      2187
weighted avg       0.91      0.92      0.91      2187

              precision    recall  f1-score   support

       False       0.92      1.00      0.96      2006
        True       0.00      0.00      0.00       181

    accuracy                           0.92      2187
   macro avg       0.46      0.50      0.48      2187
weighted avg       0.84      0.92      0.88      2187



/home/suhwan/.conda/envs/testenv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [34]:
adflist = [adf2,adf3,adf4,adf5,adf6,adf7,adf8,adf9,adf10,adf11]
prefixlist= list(range(2,12))
acc_dict= {}
for pos,prefix in enumerate(adflist):
    print('BAC aggregation encoding with prefix length %s'%(prefixlist[pos]))
    y = prefix['outcome']
    x =prefix.drop(columns=['outcome','caseid'],axis=1)
    x_train,x_test,y_train,y_test = train_test_split(x,y, test_size=0.3)

    # Deicision tree result
    dt = DecisionTreeClassifier(criterion='entropy').fit(x_train,y_train)
    y_pred = dt.predict(x_test)
    acc_dict['prefix_%s'%(str(prefixlist[pos]))] =  accuracy_score(y_test,y_pred)

import pickle as pkl

x = list(acc_dict.keys())
y = [acc_dict[x] for x in acc_dict.keys()]
with open('./result/off_dt_acc.pkl','wb') as f:
    pkl.dump([x,y],f)

BAC aggregation encoding with prefix length 2
BAC aggregation encoding with prefix length 3
BAC aggregation encoding with prefix length 4
BAC aggregation encoding with prefix length 5
BAC aggregation encoding with prefix length 6
BAC aggregation encoding with prefix length 7
BAC aggregation encoding with prefix length 8
BAC aggregation encoding with prefix length 9
BAC aggregation encoding with prefix length 10
BAC aggregation encoding with prefix length 11


In [33]:
adflist = [adf2,adf3,adf4,adf5,adf6,adf7,adf8,adf9,adf10,adf11]
prefixlist= list(range(2,12))
acc_dict= {}
for pos,prefix in enumerate(adflist):
    print('BAC aggregation encoding with prefix length %s'%(prefixlist[pos]))
    y = prefix['outcome']
    x =prefix.drop(columns=['outcome','caseid'],axis=1)
    x_train,x_test,y_train,y_test = train_test_split(x,y, test_size=0.3)

    # Deicision tree result
    dt = RandomForestClassifier(criterion='entropy').fit(x_train,y_train)
    y_pred = dt.predict(x_test)
    acc_dict['prefix_%s'%(str(prefixlist[pos]))] =  accuracy_score(y_test,y_pred)

import pickle as pkl

x = list(acc_dict.keys())
y = [acc_dict[x] for x in acc_dict.keys()]
with open('./result/off_rf_acc.pkl','wb') as f:
    pkl.dump([x,y],f)

BAC aggregation encoding with prefix length 2
BAC aggregation encoding with prefix length 3
BAC aggregation encoding with prefix length 4
BAC aggregation encoding with prefix length 5
BAC aggregation encoding with prefix length 6
BAC aggregation encoding with prefix length 7
BAC aggregation encoding with prefix length 8
BAC aggregation encoding with prefix length 9
BAC aggregation encoding with prefix length 10
BAC aggregation encoding with prefix length 11


In [ ]:
for 